In [20]:
from pathlib import Path

from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI

from dotenv import load_dotenv, find_dotenv

# from configs import *

_ = load_dotenv(find_dotenv())


PASTA_ARQUIVOS = Path('/home/saulo/PythonScripts/Asimov/ChatPdf/teste.ipynb').parent / 'arquivos'
MODEL_NAME = 'gpt-4o-mini'


def importacao_documentos():
    documentos = []
    for arquivo in PASTA_ARQUIVOS.glob('*.pdf'):
        loader = PyPDFLoader(str(arquivo))
        documentos_arquivo = loader.load()
        documentos.extend(documentos_arquivo)
    return documentos


def split_de_documentos(documentos):
    recur_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2500,
        chunk_overlap=250,
        separators=["/n\n", "\n", ".", " ", ""]
    )
    documentos = recur_splitter.split_documents(documentos)

    for i, doc in enumerate(documentos):
        doc.metadata['source'] = doc.metadata['source'].split('/')[-1]
        doc.metadata['doc_id'] = i
    return documentos


def cria_vector_store(documentos):
    embedding_model = OpenAIEmbeddings()
    vector_store = FAISS.from_documents(
        documents=documentos,
        embedding=embedding_model
    )
    return vector_store


def cria_chain_conversa(vector_store):
    chat = ChatOpenAI(model=MODEL_NAME)
    memory = ConversationBufferMemory(
        return_messages=True,
        memory_key='chat_history',
        output_key='answer'
    )
    retrivier = vector_store.as_retriever()
    chat_chain = ConversationalRetrievalChain.from_llm(
        llm=chat,
        memory=memory,
        retriever=retrivier,
        return_source_documents=True,
        verbose=True
    )
    return chat_chain


In [21]:
# import openai
# from dotenv import load_dotenv, find_dotenv

# _ = load_dotenv(find_dotenv())

# client = openai.Client()
documentos = importacao_documentos()
documentos = split_de_documentos(documentos)
vector_store = cria_vector_store(documentos)
chain = cria_chain_conversa(vector_store)

In [22]:
chain.invoke({'question':'O que é a OpenAi'})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Explorando a API da OpenAI
Asimov Academy

Explorando a API da OpenAI
Guias
Também está disponível diversos guias de boas práticas recomendadas ao utilizar os modelos da
OpenAI.
Asimov Academy 9

Explorando a API da OpenAI
A OpenAI
Nosso objetivo é explorar a API da OpenAI, a maior desenvolvedora de ferramentas de inteligência
artificial do mundo na atualidade. O modelo ChatGPT é apenas um dos vários modelos criados pela
empresa nos últimos anos. A seguir, apresentamos alguns dos principais modelos e suas funções:
•GPT-4: Trata-se de um modelo de linguagem de grande escala (LLM) multimodal, que aceita
entradas de texto ou imagem e produz texto. Ele é capaz de resolver problemas complexos com
maior pr

{'question': 'O que é a OpenAi',
 'chat_history': [HumanMessage(content='O que é a OpenAi'),
  AIMessage(content='A OpenAI é a maior desenvolvedora de ferramentas de inteligência artificial do mundo na atualidade. Ela criou vários modelos de IA, sendo o ChatGPT um deles. Seu objetivo é explorar e desenvolver tecnologias de inteligência artificial avançadas, permitindo interações e aplicações diversas através de sua API.')],
 'answer': 'A OpenAI é a maior desenvolvedora de ferramentas de inteligência artificial do mundo na atualidade. Ela criou vários modelos de IA, sendo o ChatGPT um deles. Seu objetivo é explorar e desenvolver tecnologias de inteligência artificial avançadas, permitindo interações e aplicações diversas através de sua API.',
 'source_documents': [Document(page_content='Explorando a API da OpenAI\nAsimov Academy', metadata={'source': 'Apostila - Explorando a API da OpenAI.pdf', 'page': 0, 'doc_id': 0}),
  Document(page_content='Explorando a API da OpenAI\nGuias\nTambém 